In [6]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import mediapipe as mp
import cv2
import numpy as np


In [8]:
# Calculate the angle between three points using the cosine rule
def calculate_angle(a, b, c):


    a = np.array([a.x, a.y])
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])

    radians = np.arccos(np.dot(b - a, c - b) / (np.linalg.norm(b - a) * np.linalg.norm(c - b)))
    angle = np.degrees(radians)

    return angle

In [9]:
def count_bench_press():

    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils
    cap = cv2.VideoCapture(0)

    bench_press_count = 0
    is_pressing = False

    prev_shoulder_angle = None
    prev_elbow_angle = None
    angle_threshold = 130  # Adjust this value based on your setup and preferences

    while True:
        ret, image = cap.read()

        image = cv2.flip(image, 1)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            results = pose.process(image_rgb)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            if results.pose_landmarks is not None:
                left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
                left_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
                right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
                right_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW]
                left_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]

                shoulder_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                elbow_angle = calculate_angle(left_elbow, left_shoulder, left_wrist)

                if prev_shoulder_angle is not None and prev_elbow_angle is not None:
                    if not is_pressing and shoulder_angle < angle_threshold and elbow_angle < angle_threshold:
                        is_pressing = True
                    elif is_pressing and shoulder_angle > 160 and elbow_angle > 160:
                        is_pressing = False

                prev_shoulder_angle = shoulder_angle
                prev_elbow_angle = elbow_angle

                if is_pressing:
                    cv2.putText(image, "Pressing", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                else:
                    cv2.putText(image, "Not Pressing", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                if is_pressing and shoulder_angle > 160 and elbow_angle > 160:
                    bench_press_count += 1
                    is_pressing = False

            cv2.imshow("Bench Press Counter", image)

            if cv2.waitKey(1) == ord("q"):
                break

    cap.release()
    cv2.destroyAllWindows()

    return bench_press_count


: 

In [5]:
bench_press_count = count_bench_press()